# Surface slabs

Generating surface slabs and related structures is a common task in computational materials science.

## Visualizing the WorkGraph Builder


In [ ]:
from workgraph_collections.ase.espresso.slabs import slabs_workgraph

task = slabs_workgraph.task()
task.to_html()
#
# task

## Visualizing the WorkGraph

In [ ]:
from workgraph_collections.ase.espresso.slabs import slabs_workgraph

wg = slabs_workgraph()
wg.to_html()
# wg

## Example: Al (111) and Al (100) surfaces

### Prepare the inputs and submit the workflow


In [1]:
from workgraph_collections.ase.espresso.slabs import slabs_workgraph
from ase.build import bulk
from aiida import load_profile

load_profile()

# create input structure
atoms = bulk("Al", cubic=True)
# atoms = bulk("Si")

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}
# pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "CONTROL": {"calculation": "scf"},
    "SYSTEM": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
#------------------------- Set the inputs -------------------------
wg = slabs_workgraph(atoms=atoms,
                   computer="localhost",
                   miller_indices=[[1, 1, 1], [1, 0, 0]],
                   command="mpirun -np 4 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kspacing=0.10,
                   metadata=metadata,
                   calc_surface_energy=True,
                   )
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=300)



WorkGraph process created, PK: 156499


<WorkChainNode: uuid: 52e39e02-c48b-4a25-b5fd-6179ffb70d98 (pk: 156499) (aiida.workflows:workgraph.engine)>

## Relaxed slabs

In [3]:
print("Surface   Energy")
for key, E in wg.tasks["get_surface_energy"].outputs.result.value.items():
    print(key, E)

print("Surfae   Atoms")
for key, slab in wg.tasks["relax_slabs"].outputs.structures.value.items():
    print(key, slab.value)



Surface   Energy
slab100 1.4816161652404
slab111 1.4816131295258
Surfae   Atoms
slab100 Atoms(symbols='Al3', pbc=True, cell=[[2.8794445061349, 0.0, 0.0], [1.4397222530675, 2.4936720911004, 0.0], [0.0, 0.0, 14.702113144751]])
slab111 Atoms(symbols='Al3', pbc=True, cell=[[2.8794445061349, 0.0, 0.0], [1.4397222530675, 2.4936720911004, 0.0], [0.0, 0.0, 14.702113144751]])
